In [ ]:
import numpy as np
X = np.load("pneumonia_images.npy")
y = np.load("pneumonia_labels.npy")

Visualizziamo alcune immagini del dataset, target = 1 => Polmonite, target = 0 => Negativo

In [ ]:
print("Number of images:", X.shape[0])
positive = 0
negative = 0

for i in range(X.shape[0]):
    if(y[i]==1):
        positive += 1
    if(y[i]==0):
        negative += 1

print("Number of positive cases:", positive)
print("Number of negative cases:", negative)

In [ ]:

import matplotlib.pyplot as plt


# Visualizza alcune immagini
num_images_to_display = 5

for i in range(num_images_to_display):
    # Seleziona un'immagine casuale
    index = np.random.randint(0, len(X))
    image = X[index]
    label = y[index]

    # Visualizza l'immagine con la sua etichetta
    plt.subplot(1, num_images_to_display, i + 1)
    plt.imshow(image, cmap='gray')
    plt.title(f"Label: {label}")
    plt.axis('off')

plt.show()

Il dataset è sbilanciato verso le immagini con polmonite:

Circa il 74.2% delle immagini rappresentano casi di polmonite.
Circa il 25.8% delle immagini rappresentano casi negativi.

Andiamo a suddividere il train set tra training e test, utilizzando stratify per mantenere proporzione tra le classi

In [ ]:
from sklearn.model_selection import train_test_split

# Suddivisione in set di addestramento e set di validazione
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

A questo punto andiamo a dividere il Training Set in Training e Validation, mantenendo la stessa proporzione

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=42, stratify=y_train)

Vado a normalizzare i dati dopo aver suddiviso Training, Validation e Test per evitare Data Leakage

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_train_reshaped = X_train.reshape(X_train.shape[0], -1)
X_val_reshaped = X_valid.reshape(X_valid.shape[0], -1)
X_test_reshaped = X_test.reshape(X_test.shape[0], -1)

X_train_normalized = scaler.fit_transform(X_train_reshaped).reshape(X_train.shape)
X_val_normalized = scaler.transform(X_val_reshaped).reshape(X_valid.shape)
X_test_normalized = scaler.transform(X_test_reshaped).reshape(X_test.shape)

print(f"Normalized training set size: {X_train_normalized.shape}")
print(f"Normalized validation set size: {X_val_normalized.shape}")
print(f"Normalized test set size: {X_test_normalized.shape}")

Essendo le classi sbilanciate, preferiamo una metrica come F1 rispetto a accuracy

In [ ]:
from tensorflow.keras import backend as K

# Funzione per calcolare l'F1-score
def f1_score(y_true, y_pred):
    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    f1 = 2 * ((precision * recall) / (precision + recall + K.epsilon()))
    return f1

Definiamo una funzione per la stampa dei vari grafici

In [ ]:
import matplotlib.pyplot as plt

def graphics(history):
    plt.figure(figsize=(12, 4))

    # Plot della perdita
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    # Plot delle metriche
    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.plot(history.history['f1_score'], label='Training F1 Score')
    plt.plot(history.history['val_f1_score'], label='Validation F1 Score')
    plt.title('Training and Validation Metrics')
    plt.xlabel('Epochs')
    plt.ylabel('Metrics')
    plt.legend()

    plt.show()

Andiamo a eseguire il Training tramite Convolutional NN, utilizziamo una semplice CNN composta da 2 strati convoluzionali e uno strato Dense da 128 unità per la classificazione binaria, quest'ultima realizzata con un livello Dense di 1 unità con funzione di attivazione sigmoid

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [ ]:
cnn1 = Sequential([
    Conv2D(8, kernel_size=(3, 3), activation='relu',padding='same',input_shape=(28,28,1)),
    MaxPooling2D(pool_size=(2, 2),strides=2),
    Conv2D(16, kernel_size=(5, 5), activation='relu'),
    MaxPooling2D(pool_size=(2, 2),strides=2),
    Flatten(),
    Dense(120, activation='relu'),
    Dense(84, activation='relu'),
    Dense(1, activation='sigmoid')
])


In [ ]:
# Stampa una rappresentazione del modello
cnn1.summary()

In [ ]:
# Compilazione del modello
cnn1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', f1_score])

In [ ]:
history = cnn1.fit(X_train_normalized, y_train, epochs=50, batch_size=32, validation_data=(X_val_normalized, y_valid))

In [ ]:
graphics(history)

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

y_pred_prob = cnn1.predict(X_val_normalized)
y_pred = np.round(y_pred_prob).astype(int)

cm = confusion_matrix(y_valid, y_pred)

disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap=plt.cm.Blues)
plt.show()

In [ ]:
test_loss, test_accuracy = model.evaluate(X_test_normalized, y_test, batch_size=32)

print(f"Loss sul set di test: {test_loss}")
print(f"Accuracy sul set di test: {test_accuracy}")

Il modello, dalla curva riguardante la Validation Loss, mostra Overfitting, proviamo ad usare tecniche per ridurlo

Tecniche usate:
    -Dropout
    -Early Stopping

Aggiungiamo inoltre Batch Normalization.
In questo caso andiamo a stabilire dei valori base, successivamente faremo del tuning degli hyperparameters

In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2

In [ ]:
# Definizione del modello
cnn1 = Sequential([
    Conv2D(8, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2)),
    BatchNormalization(),
    Dropout(0.25),
    
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    BatchNormalization(),
    Dropout(0.25),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    BatchNormalization(),
    Dropout(0.25),
    
    Flatten(),
    Dense(256, activation='relu', kernel_regularizer=l2(0.01)),
    BatchNormalization(),
    Dropout(0.5),
    
    Dense(128, activation='relu', kernel_regularizer=l2(0.01)),
    BatchNormalization(),
    Dropout(0.5),
    
    Dense(1, activation='sigmoid')
])


In [ ]:
# Compilazione del modello
cnn1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', f1_score])

# Definizione del callback EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

cnn1.summary()

In [ ]:
# Addestramento del modello
history2 = cnn1.fit(X_train_normalized, y_train, 
                    epochs=50, 
                    batch_size=32, 
                    validation_data=(X_val_normalized, y_valid), 
                    callbacks=[early_stopping])

In [ ]:
graphics(history2)

Abbiamo un buon miglioramento della curva della validation loss, mentre abbiamo valori di Validation F1 Score e Validation Accuracy leggermente fluttuanti nelle varie epoche. Vediamo la Confusion Matrix:

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

y_pred_prob = cnn1.predict(X_val_normalized)
y_pred = np.round(y_pred_prob).astype(int)

cm = confusion_matrix(y_valid, y_pred)

disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap=plt.cm.Blues)
plt.show()

In [ ]:
test_loss, test_accuracy, test_f1 = cnn1.evaluate(X_test_normalized, y_test, batch_size=32)

print(f"Loss sul set di test: {test_loss}")
print(f"Accuracy sul set di test: {test_accuracy}")
print(f"F1 Score sul set di test: {f1_metric}")

Inseriamo Data Augmentation.
Inanzitutto adattiamo i dati per avere 4 dimensioni, input richiesto per Data Augmentation

In [ ]:
X_train = np.expand_dims(X_train_normalized, axis=-1)
X_val = np.expand_dims(X_val_normalized, axis=-1)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    vertical_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    horizontal_flip=False,  # Non riflettere lungo l'asse x per immagini mediche
    fill_mode='nearest'
)

# Definisci il generatore di dati per il set di validazione senza data augmentation
val_datagen = ImageDataGenerator()

# Assicurati di adattare i generatori ai tuoi dati
train_generator = train_datagen.flow(X_train, y_train, batch_size=32)
val_generator = val_datagen.flow(X_val, y_valid, batch_size=32)

In [ ]:
# Visualizza alcune immagini originali
num_images_to_display = 5
plt.figure(figsize=(15, 5))

for i in range(num_images_to_display):
    # Seleziona un'immagine casuale
    index = np.random.randint(0, len(X_train))
    original_image = X_train[index]
    original_label = y_train[index]

    # Visualizza l'immagine originale con la sua etichetta
    plt.subplot(2, num_images_to_display, i + 1)
    plt.imshow(original_image, cmap='gray')
    plt.title(f"Original Label: {original_label}")
    plt.axis('off')

# Visualizza alcune immagini generate
for i in range(num_images_to_display):
    augmented_data = next(train_generator)
    augmented_image = augmented_data[0][0]  # Ottieni l'immagine generata
    augmented_label = augmented_data[1][0]  # Ottieni l'etichetta generata

    # Visualizza l'immagine generata con la sua etichetta
    plt.subplot(2, num_images_to_display, num_images_to_display + i + 1)
    plt.imshow(augmented_image, cmap='gray')
    plt.title(f"Augmented Label: {augmented_label}")
    plt.axis('off')

plt.show()

In [ ]:
cnn2 = Sequential([
    Conv2D(16, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2)),
    BatchNormalization(),
    Dropout(0.25),
    
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    BatchNormalization(),
    Dropout(0.25),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    BatchNormalization(),
    Dropout(0.25),
    
    Flatten(),
    Dense(256, activation='relu', kernel_regularizer=l2(0.01)),
    BatchNormalization(),
    Dropout(0.5),
    
    Dense(128, activation='relu', kernel_regularizer=l2(0.01)),
    BatchNormalization(),
    Dropout(0.5),
    
    Dense(1, activation='sigmoid')
])

# Compilazione del modello
cnn2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy',f1_score])

# Definizione del callback EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Addestramento del modello con data augmentation
history = cnn2.fit(
    train_generator,
    steps_per_epoch=len(X_train_normalized) // 32,
    epochs=50,
    validation_data=val_generator,
    validation_steps=len(X_val_normalized) // 32,
    callbacks=[early_stopping]
)

In [ ]:
graphics(history)

In [ ]:
test_loss, test_accuracy, test_f1 = cnn2.evaluate(X_test_normalized, y_test, batch_size=64)

print(f"Loss sul set di test: {test_loss}")
print(f"Accuracy sul set di test: {test_accuracy}")
print(f"F1 Score sul set di test: {f1_metric}")

Proviamo tuning hyperparameters

In [ ]:
pip install keras-tuner --upgrade